# Evaluate the SWPP Wind Forecast Fidelity During the June 2023 Lull Event


In [1]:
# Start by importing the packages we need:
import os
import glob
import datetime

import pandas as pd
import matplotlib.pyplot as plt


## Set the Directory Structure


In [2]:
# Identify the data input and image output directories:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/spp_wind_forecast/data/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/spp_wind_forecast/data/plots/'


## Format the Forecast Data


In [3]:
# Read in the .csv file and replace missing values with nan:
forecast_df = pd.read_csv((data_input_dir + '20230531_Forecast.csv'))

# Rename the a few variables for simplicity:
forecast_df.rename(columns={'STLF': 'ST_Load_MWh',
                            'MTLF': 'MT_Load_MWh',
                            'STWF': 'ST_Wind_MWh',
                            'MTWF': 'MT_Wind_MWh'}, inplace=True)

# Convert the time to a datetime variable:
forecast_df['Time_UTC'] = pd.to_datetime(forecast_df['GMTIntervalEnd'])

# Only keep the columns that are needed:
forecast_df = forecast_df[['Time_UTC', 'ST_Load_MWh', 'MT_Load_MWh', 'ST_Wind_MWh', 'MT_Wind_MWh']].copy()

# Drop rows with missing values:
forecast_df = forecast_df.dropna(thresh=2)

# Return the formatted dataframe:
forecast_df


,Time_UTC,ST_Load_MWh,MT_Load_MWh,ST_Wind_MWh,MT_Wind_MWh
9,2023-06-07 21:00:00,NaN,39373.901,NaN,3336.00
21,2023-06-07 20:00:00,NaN,38621.401,NaN,3173.52
33,2023-06-07 19:00:00,NaN,37613.888,NaN,2998.83
45,2023-06-07 18:00:00,NaN,36114.435,NaN,2793.63
57,2023-06-07 17:00:00,NaN,34557.580,NaN,2623.93
...,...,...,...,...,...
2299,2023-05-30 22:10:00,40877.191,NaN,6951.26,NaN
2300,2023-05-30 22:05:00,40868.415,NaN,6792.71,NaN
2301,2023-05-30 22:00:00,40856.254,40768.896,6746.81,7077.82
2302,2023-05-30 21:55:00,40865.289,NaN,6812.30,NaN


## Format the Observed Data


In [4]:
# Read in the data from the EIA-930 Excel file:
obs_df = pd.read_excel((data_input_dir + '20230609_EIA930.xlsx'), sheet_name='Data')

# Convert the time to a datetime variable:
obs_df['Time_UTC'] = pd.to_datetime(obs_df['UTC time'])

# Rename the a few variables for simplicity:
obs_df.rename(columns={'D': 'Obs_Load_MWh',
                       'NG: WND': 'Obs_Wind_MWh'}, inplace=True)

# Only keep the columns that are needed:
obs_df = obs_df[['Time_UTC', 'Obs_Load_MWh', 'Obs_Wind_MWh']].copy()

# Return the formatted dataframe:
obs_df


,Time_UTC,Obs_Load_MWh,Obs_Wind_MWh
0,2023-05-30 00:00:00,35521,5892
1,2023-05-30 01:00:00,34682,6774
2,2023-05-30 02:00:00,33580,9394
3,2023-05-30 03:00:00,32693,12164
4,2023-05-30 04:00:00,30831,14299
...,...,...,...
241,2023-06-09 01:00:00,39598,2364
242,2023-06-09 02:00:00,37875,2681
243,2023-06-09 03:00:00,36499,3283
244,2023-06-09 04:00:00,34241,4399


## Merge the Forecast and Observed Data Together


In [5]:
# Merge the two dataframes together based on common UTC times:
merged_df = forecast_df.merge(obs_df, on=['Time_UTC'])

# Calculate the errors:
merged_df['ST_Load_Error_MWh'] = (merged_df['Obs_Load_MWh'] - merged_df['ST_Load_MWh']).round(2)
merged_df['MT_Load_Error_MWh'] = (merged_df['Obs_Load_MWh'] - merged_df['MT_Load_MWh']).round(2)
merged_df['ST_Wind_Error_MWh'] = (merged_df['Obs_Wind_MWh'] - merged_df['ST_Wind_MWh']).round(2)
merged_df['MT_Wind_Error_MWh'] = (merged_df['Obs_Wind_MWh'] - merged_df['MT_Wind_MWh']).round(2)

merged_df


,Time_UTC,ST_Load_MWh,MT_Load_MWh,ST_Wind_MWh,MT_Wind_MWh,Obs_Load_MWh,Obs_Wind_MWh,ST_Load_Error_MWh,MT_Load_Error_MWh,ST_Wind_Error_MWh,MT_Wind_Error_MWh
0,2023-06-07 21:00:00,NaN,39373.901,NaN,3336.00,42602,2576,NaN,3228.10,NaN,-760.00
1,2023-06-07 20:00:00,NaN,38621.401,NaN,3173.52,41678,1423,NaN,3056.60,NaN,-1750.52
2,2023-06-07 19:00:00,NaN,37613.888,NaN,2998.83,40399,797,NaN,2785.11,NaN,-2201.83
3,2023-06-07 18:00:00,NaN,36114.435,NaN,2793.63,38785,593,NaN,2670.57,NaN,-2200.63
4,2023-06-07 17:00:00,NaN,34557.580,NaN,2623.93,36784,529,NaN,2226.42,NaN,-2094.93
...,...,...,...,...,...,...,...,...,...,...,...
187,2023-05-31 02:00:00,36161.368,36797.324,13179.01,11459.38,36654,12109,492.63,-143.32,-1070.01,649.62
188,2023-05-31 01:00:00,37492.823,38349.377,10733.92,10622.33,38299,10579,806.18,-50.38,-154.92,-43.33
189,2023-05-31 00:00:00,39168.834,39910.145,9891.59,9093.07,39758,9520,589.17,-152.14,-371.59,426.93
190,2023-05-30 23:00:00,40352.520,40503.392,8300.92,7210.82,40668,7679,315.48,164.61,-621.92,468.18
